In [3]:
import sys
sys.path.append('../')

import torch
import numpy as np
import json
from tqdm import tqdm
from lib.service import SamplesService

In [4]:
def decode_int64_bitset(x):
    """
    Convert a 64-bit integer into a 64-element float tensor
    """
    masks = 2 ** torch.arange(64, dtype=torch.int64, device='cuda')
    expanded = torch.bitwise_and(x.unsqueeze(-1), masks).ne(0).to(torch.float32)
    return expanded

intermediate_scale = 1 / 64
output_scale = 1 / 9600

class ChessModel(torch.nn.Module):

    def __init__(self, num_features):
        super(ChessModel, self).__init__()
        
        self.linear1 = torch.nn.Linear(num_features, 1024)
        self.linear2 = torch.nn.Linear(1024, 64)
        self.linear3 = torch.nn.Linear(64, 64)
        self.output = torch.nn.Linear(64, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.clamp(x, 0.0, 1.0) # Clipped ReLU
        x = self.linear2(x)
        x = torch.clamp(x, 0.0, 1.0) # Clipped ReLU
        x = self.linear3(x)
        x = torch.clamp(x, 0.0, 1.0) # Clipped ReLU
        return self.output(x)
    
    def _clip_weights(self):
        intermediate_clip = 127 * intermediate_scale
        output_clip = 127*127 * output_scale

        self.linear1.weight.data.clamp_(-intermediate_clip, intermediate_clip)
        self.linear2.weight.data.clamp_(-intermediate_clip, intermediate_clip)
        self.linear3.weight.data.clamp_(-intermediate_clip, intermediate_clip)
        self.output.weight.data.clamp_(-output_clip, output_clip)

    def _layer_json(self, layer: torch.nn.Linear, scale_value: float, activation: str | None = None):
        weights = layer.weight.data.cpu().numpy()
        bias = layer.bias.data.cpu().numpy()

        qweights = np.round(weights / scale_value).astype('int8')
        qbias = np.round(bias / (1 / 32_)).astype('int32')

        obj = {
            "num_inputs": layer.in_features,
            "num_outputs": layer.out_features,
            "weights": qweights.reshape(-1).tolist(),
            "bias": qbias.reshape(-1).tolist()
        }

        if activation is not None:
            obj["activation"] = activation

        return obj

    def to_json(self):
        """
        Returns the layers of the model as a list of dictionaries
        """
        layers = []
        layers.append(self._layer_json(self.linear1, intermediate_scale, activation='clip'))
        layers.append(self._layer_json(self.linear2, intermediate_scale, activation='clip'))
        layers.append(self._layer_json(self.linear3, intermediate_scale, activation='clip'))
        layers.append(self._layer_json(self.output, output_scale))
        return layers

#testmodel = ChessModel(768)
#testmodel.load_state_dict(torch.load('/mnt/c/Users/mlomb/Desktop/Tesis/cs-master-thesis/notebooks/runs/20240308_130550_eval_basic_4096/models/400.pth'))
#testmodel.cuda()

#X, y = samples_service.next_batch()
#X = decode_int64_bitset(X)
#X = X.reshape(-1, NUM_FEATURES)
#print(testmodel(X) / 356 * 100, y)

#import json
#with open('model.json', 'w') as f:
#    f.write(json.dumps(testmodel.to_json()))

In [3]:
class PQRLoss(torch.nn.Module):
    def __init__(self):
        super(PQRLoss, self).__init__()

    def forward(self, pred):
        pred = pred.reshape(-1, 3)
        
        p = pred[:,0]
        q = pred[:,1]
        r = pred[:,2]
        
        a = -torch.mean(torch.log(torch.sigmoid(r - q)))
        b = torch.mean(torch.square(p + q))

        loss = a + b

        return loss

class EvalLoss(torch.nn.Module):
    def __init__(self):
        super(EvalLoss, self).__init__()

    def forward(self, output, target):
        # since we are clipping the layers, we need to scale the output so it can reach higher values
        output = output * 600.0

        scaling = 356.0

        # scale CP score to engine units [-10_000, 10_000]
        target = target * scaling / 100.0

        # targets are in CP-space change it to WDL-space [0, 1]
        wdl_model = torch.sigmoid(output / scaling)
        wdl_target = torch.sigmoid(target / scaling)

        loss = torch.pow(torch.abs(wdl_model - wdl_target), 2.5)

        return loss.mean()

In [4]:
import os
import math
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from glob import glob

EPOCHS = 100000
BATCHES_PER_EPOCH = 1000
BATCH_SIZE = 4096

FEATURE_SET = "basic"
NUM_FEATURES = 768
METHOD = "eval"

if METHOD == "pqr":
    X_SHAPE = (BATCH_SIZE, 3, NUM_FEATURES // 64)
    Y_SHAPE = (BATCH_SIZE, 0)
    INPUTS = glob("/mnt/d/datasets/pqr-1700/*.csv")
    loss_fn = PQRLoss()
elif METHOD == "eval":
    X_SHAPE = (BATCH_SIZE, NUM_FEATURES // 64)
    Y_SHAPE = (BATCH_SIZE, 1)
    INPUTS = glob("/mnt/d/datasets/eval/*.csv")
    loss_fn = EvalLoss()

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
folder = f'runs/{timestamp}_{METHOD}_{FEATURE_SET}_{BATCH_SIZE}'
os.makedirs(f'{folder}/models', exist_ok=True)

samples_service = SamplesService(x_shape=X_SHAPE, y_shape=Y_SHAPE, inputs=INPUTS, feature_set=FEATURE_SET, method=METHOD)
chessmodel = ChessModel(num_features=NUM_FEATURES)
chessmodel.cuda()

#for i in tqdm(range(1000000)):
#    a = samples_service.next_batch()

optimizer = torch.optim.Adam(chessmodel.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', threshold=0.0001, factor=0.7, patience=10)
writer = SummaryWriter(folder)

# @torch.compile # 30% speedup
def train_step(X, y):
    # Clear the gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = chessmodel(X)

    # Compute the loss
    loss = loss_fn(outputs, y)
    loss.backward()

    # Update the parameters
    optimizer.step()

    chessmodel._clip_weights()

    return loss

# Make sure gradient tracking is on
chessmodel.train()

for epoch in range(EPOCHS):
    avg_loss = 0.0

    for _ in tqdm(range(BATCHES_PER_EPOCH), desc=f'Epoch {epoch}'):
        X, y = samples_service.next_batch()
    
        # expand bitset
        X = decode_int64_bitset(X)
        X = X.reshape(-1, NUM_FEATURES)

        loss = train_step(X, y)
        avg_loss += loss.item()

        if math.isnan(avg_loss):
            raise Exception("Loss is NaN, exiting")

    avg_loss /= BATCHES_PER_EPOCH

    # Step the scheduler
    scheduler.step(avg_loss)

    writer.add_scalar('Train/loss', avg_loss, epoch)
    writer.add_scalar('Train/lr', scheduler._last_lr[0], epoch) # get_last_lr()
    writer.add_scalar('Params/mean-l1', torch.mean(chessmodel.linear1.weight), epoch)
    writer.add_scalar('Params/mean-l2', torch.mean(chessmodel.linear2.weight), epoch)
    writer.add_scalar('Params/mean-l3', torch.mean(chessmodel.linear3.weight), epoch)
    writer.add_scalar('Params/mean-l4', torch.mean(chessmodel.output.weight), epoch)
    for name, param in chessmodel.named_parameters():
        writer.add_histogram(name, param, epoch)
    writer.flush()

    # save model
    model_path = f'{folder}/models/{epoch}'
    torch.save(chessmodel.state_dict(), f'{model_path}.pth')
    with open(f'{model_path}.json', 'w') as f:
        model = {
            "config": {
                "batches_per_epoch": BATCHES_PER_EPOCH,
                "batch_size": BATCH_SIZE,

                "feature_set": FEATURE_SET,
                "num_features": NUM_FEATURES,
                "method": METHOD,
            },
            "train": {
                "epoch": epoch,
                "loss": avg_loss
            },
            "layers": chessmodel.to_json()
        }

        f.write(json.dumps(model))

2024-03-10 22:06:22.432864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 22:06:22.433007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 22:06:22.518002: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 22:06:22.681179: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 22:06:24.737950: W tensorflow/compiler/tf2

KeyboardInterrupt: 

In [ ]:
a = list(chessmodel.output.parameters())[0].cpu().detach().numpy()

import seaborn as sns

# plot distribution
sns.histplot(a.flatten(), kde=True)


In [51]:
testmodel = ChessModel(768)
testmodel.load_state_dict(torch.load('/mnt/c/Users/mlomb/Desktop/Tesis/cs-master-thesis/notebooks/runs/20240310_220627_eval_basic_4096/models/0.pth'))

input = np.array([
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0,
])

fl = testmodel.linear1
r = fl(torch.tensor(input, dtype=torch.float32).reshape(-1, 768))

np.set_printoptions(threshold=sys.maxsize, suppress=True)
print(np.round(fl.bias.detach().numpy()*127*64))
print(r.detach().numpy()*100)


[  69.   -3.  -43.  112.   76.  104.  244.   74. -157.  367.  139.   79.
   36.  -73.  -90.  -24. -132.  201.  144.  -84. -111.  239.   96.  496.
  347.   19.   61.  -10.  236.  225.  175.   91.  287.  366.  322.   23.
    7.  372. -213.  -47.   21.  322.  271.  144.  294. -154. -190.   79.
   47. -273.  160.   66.   11.   94.  196.  378. -135. -216.  226. -232.
   31. -166.  -49.  -91.    5. -143.   82.   66.    3.  205.   89.  -66.
  164.   -7.   98.  261.  172. -207.  -35.   97.  312.   33.  294. -146.
  -58.   61.  285.  200.   30.   24.   76. -128.  214.   58.  198.   33.
   -1.  217.  196.  138.  -50.  326.  250.  261.  299.  -61.  183.   55.
   40.   -5.  306.  163.  325. -109. -111. -189.  345.  195. -226.  192.
 -162.   95.  219.  310.  325.  143.  145.  170.  170. -142.  362.  260.
  301.  174.  291.  222. -119.  271.  110.  -61. -174.  415.  241.   95.
   79. -155.  -10.  187.   63.   -8.  125.  130.  117.  373.  -95.  215.
   12.  118.  124.   19.    1.  400.  -37.  -82.  1